In [1]:
import polars as pl

In [ ]:
# load holesky data
holesky_blocks = pl.scan_parquet(
    'data/blocks.parquet').rename({'number': 'block_number'}).select('block_number', 'timestamp', 'extra_data', 'blob_gas_used', 'excess_blob_gas', 'base_fee_per_gas').with_columns(pl.from_epoch('timestamp'))
holesky_txs = pl.scan_parquet('data/transactions.parquet').select('block_number', 'hash',
                                                                  'to', 'from', 'type', 'gas', 'gas_used', 'gas_price', 'effective_gas_price', 'max_priority_fee_per_gas', 'max_fee_per_gas')

holesky_blocks_txs = (holesky_txs.join(
    holesky_blocks, on='block_number', how='left')
    .fill_null(strategy="forward")
    .with_columns(
        (pl.col("base_fee_per_gas") * pl.col("gas_used")).alias("base_tx_fee_eth"),
        (pl.col("effective_gas_price") - pl.col("base_fee_per_gas")).alias(
            "priority_fee_gas"
        )
)
    .with_columns(
        # have to perform priority fee calculation in this column
        (((pl.col("effective_gas_price") - pl.col("base_fee_per_gas"))
          * pl.col("gas_used")) / 10**18).alias("priority_tx_fee_eth"),
        # unit calculations for gwei and eth values
        (pl.col("base_tx_fee_eth") / 10**18).alias("base_tx_fee_eth"),
        (pl.col("priority_fee_gas") / 10**9).alias("priority_fee_gas"),
        (pl.col("base_fee_per_gas") / 10**9).alias("base_fee_per_gas")
)
)